In [36]:
!pip install pandas numpy bokeh geopy openpyxl



In [37]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()



Loading BokehJS ...

In [39]:
df = pd.read_csv(r"D:\Aditya\IITG C and A\Capstone project\dataset.csv")
df.columns = df.columns.str.strip()

# Combine and convert Time
df['Time'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True, errors='coerce')
df.dropna(subset=['Time'], inplace=True)

# Rename for model compatibility
df.rename(columns={
    'ID': 'ParkingLotID',
    'TrafficConditionNearby': 'Traffic',
    'IsSpecialDay': 'SpecialDay'
}, inplace=True)

# Encode categorical fields
traffic_map = {'low': 1, 'medium': 5, 'high': 10}
df['Traffic'] = df['Traffic'].str.lower().map(traffic_map).fillna(5)
df['VehicleType'] = df['VehicleType'].astype(str).str.strip().str.lower()


In [40]:
base_price = 10
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
lot_ids = df['ParkingLotID'].unique()
lot_focus = lot_ids[0]
location_map = df.groupby('ParkingLotID')[['Latitude', 'Longitude']].first().to_dict('index')
price_tracker = {lot_focus: [base_price]}

In [46]:
import pandas as pd

df = pd.read_csv("cleaned_dataset.csv")
df['Time'] = pd.to_datetime(df['Time'])  # Ensure datetime format


In [49]:
print(df[['ParkingLotID']].drop_duplicates().head(10))
print("Unique lot count:", df['ParkingLotID'].nunique())


   ParkingLotID
0             0
1             1
2             2
3             3
4             4
5             5
6             6
7             7
8             8
9             9
Unique lot count: 18368


In [51]:
df[['SystemCodeNumber']].value_counts().head(10)


SystemCodeNumber
BHMBCCMKT01         1312
BHMBCCTHL01         1312
BHMEURBRD01         1312
BHMMBMMBX01         1312
BHMNCPHST01         1312
BHMNCPNST01         1312
Broad Street        1312
Others-CCCPS105a    1312
Others-CCCPS119a    1312
Others-CCCPS135a    1312
Name: count, dtype: int64

In [54]:
import pandas as pd

# Load original
df = pd.read_csv(r"D:\Aditya\IITG C and A\Capstone project\dataset.csv")
df.columns = df.columns.str.strip()

# Combine date and time
df['Time'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True, errors='coerce')
df.dropna(subset=['Time'], inplace=True)

# Rename the correct ParkingLotID
df.rename(columns={
    'SystemCodeNumber': 'ParkingLotID',
    'TrafficConditionNearby': 'Traffic',
    'IsSpecialDay': 'SpecialDay'
}, inplace=True)

# Drop the original row-based ID
df.drop(columns=['ID'], inplace=True)

# Encode traffic
traffic_map = {'low': 1, 'medium': 5, 'high': 10}
df['Traffic'] = df['Traffic'].str.lower().map(traffic_map).fillna(5)

# Clean vehicle type
df['VehicleType'] = df['VehicleType'].astype(str).str.strip().str.lower()

# Save cleaned version
df.to_csv("cleaned_dataset_v2.csv", index=False)
print("✅ Cleaned file saved as 'cleaned_dataset_v2.csv'")


✅ Cleaned file saved as 'cleaned_dataset_v2.csv'


In [55]:
import pandas as pd

df = pd.read_csv("cleaned_dataset_v2.csv")
df['Time'] = pd.to_datetime(df['Time'])  # Just to be safe


In [56]:
print(df['ParkingLotID'].value_counts())


ParkingLotID
BHMBCCMKT01         1312
BHMBCCTHL01         1312
BHMEURBRD01         1312
BHMMBMMBX01         1312
BHMNCPHST01         1312
BHMNCPNST01         1312
Broad Street        1312
Others-CCCPS105a    1312
Others-CCCPS119a    1312
Others-CCCPS135a    1312
Others-CCCPS202     1312
Others-CCCPS8       1312
Others-CCCPS98      1312
Shopping            1312
Name: count, dtype: int64


In [57]:
lot_focus = "BHMBCCMKT01"  # Replace with any top lot from above
df_lot = df[df['ParkingLotID'] == lot_focus].copy()


In [58]:
base_price = 10
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

from geopy.distance import geodesic
import numpy as np

location_map = df.groupby('ParkingLotID')[['Latitude', 'Longitude']].first().to_dict('index')
price_tracker = {lot_focus: [base_price]}


In [59]:
def model1_linear(occupancy, capacity, price_prev, alpha=2.0):
    return round(price_prev + alpha * (occupancy / capacity), 2)

def model2_demand_based(row, price_prev, lambda_factor=0.5):
    occ = row['Occupancy'] / row['Capacity']
    queue = row['QueueLength'] / 10
    traffic = row['Traffic'] / 10
    special = row['SpecialDay']
    vehicle = vehicle_weights.get(row['VehicleType'], 1.0)
    demand = 1.5 * occ + 1.0 * queue - 0.5 * traffic + 0.75 * special + 0.3 * vehicle
    norm = max(0, min(demand / 5.0, 1))
    return round(min(max(base_price * (1 + lambda_factor * norm), base_price * 0.5), base_price * 2), 2)

def model3_competitive(lot_id, current_price, occupancy, location_map):
    lot_location = location_map[lot_id]
    similar_lots = []
    for other_lot, coords in location_map.items():
        if other_lot == lot_id:
            continue
        dist = geodesic((lot_location['Latitude'], lot_location['Longitude']),
                        (coords['Latitude'], coords['Longitude'])).meters
        if dist <= 500:
            similar_lots.append(price_tracker.get(other_lot, [base_price])[-1])
    if not similar_lots:
        return current_price
    avg_price = np.mean(similar_lots)
    if occupancy >= 0.95:
        return round(min(current_price, avg_price - 0.5), 2)
    elif occupancy <= 0.2:
        return round(max(current_price, avg_price + 0.5), 2)
    return round((current_price + avg_price) / 2, 2)


In [60]:
price_history = []

for i, row in df_lot.iterrows():
    time_point = pd.to_datetime(row['Time'])
    occ_ratio = row['Occupancy'] / row['Capacity']
    prev_price = price_tracker[lot_focus][-1]

    p1 = model1_linear(row['Occupancy'], row['Capacity'], prev_price)
    p2 = model2_demand_based(row, p1)
    p3 = model3_competitive(lot_focus, p2, occ_ratio, location_map)

    price_tracker[lot_focus].append(p3)
    price_history.append((time_point, p3))


In [61]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

if price_history:
    df_plot = pd.DataFrame(price_history, columns=["Time", "Price"])
    df_plot["Time"] = pd.to_datetime(df_plot["Time"])
    source = ColumnDataSource(df_plot)

    p = figure(title=f"Price Trend - Lot {lot_focus}",
               x_axis_type='datetime', width=800, height=400)
    p.line(x='Time', y='Price', source=source, line_width=2, color='green')
    p.yaxis.axis_label = "Price ($)"
    p.xaxis.axis_label = "Time"
    show(p)
else:
    print("⚠️ No data to plot.")


Loading BokehJS ...